In [ ]:

import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

class SiameseNet(nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 53 * 53, 512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )
        self.out = nn.Sequential(
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def get_embedding(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def forward(self, x1, x2):
        emb1 = self.get_embedding(x1)
        emb2 = self.get_embedding(x2)
        dist = torch.abs(emb1 - emb2)
        out = self.out(dist)
        return out


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Unzip using !unzip command
!unzip "/content/drive/MyDrive/Comys_Hackathon5.zip" -d "/content/Comys_Hackathon5"

Streaming output truncated to the last 5000 lines.
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Luc_Montagnier/distortion/Luc_Montagnier_0001_lowlight.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Luc_Montagnier/distortion/Luc_Montagnier_0001_foggy.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Luc_Montagnier/distortion/Luc_Montagnier_0001_blurred.jpg  
   creating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_sunny.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_resized.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_rainy.jpg  
  inflating: /content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/Lucy_Liu/distortion/Lucy_Liu_0005_noisy.jpg  
  inflating

In [ ]:

class FacePairDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.identity_folders = sorted([f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))])
        self.samples = []

        for person in self.identity_folders:
            ref_path = os.path.join(root_dir, person)
            ref_img = [i for i in os.listdir(ref_path) if i.endswith('.jpg')][0]
            ref_img_path = os.path.join(ref_path, ref_img)
            distorted_folder = os.path.join(ref_path, 'distortion')
            distorted_imgs = [os.path.join(distorted_folder, f) for f in os.listdir(distorted_folder)]

            for img in distorted_imgs:
                self.samples.append((ref_img_path, img, 1))

            neg_candidates = [p for p in self.identity_folders if p != person]
            neg_person = random.choice(neg_candidates)
            neg_path = os.path.join(root_dir, neg_person)
            neg_img = [i for i in os.listdir(neg_path) if i.endswith('.jpg')][0]
            self.samples.append((ref_img_path, os.path.join(neg_path, neg_img), 0))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.samples[idx]
        img1 = Image.open(img1_path).convert('RGB')
        img2 = Image.open(img2_path).convert('RGB')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)


In [ ]:

def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for img1, img2, label in loader:
        img1, img2, label = img1.to(device), img2.to(device), label.to(device).unsqueeze(1)
        optimizer.zero_grad()
        out = model(img1, img2)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for img1, img2, label in loader:
            img1, img2 = img1.to(device), img2.to(device)
            out = model(img1, img2)
            pred = (out > 0.5).float().cpu().numpy()
            y_pred.extend(pred)
            y_true.extend(label.numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    return acc, f1


In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_path = '/content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train'
val_path = '/content/Comys_Hackathon5/Comys_Hackathon5/Task_B/val'

train_dataset = FacePairDataset(train_path, transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

model = SiameseNet().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(5):
    loss = train(model, train_loader, criterion, optimizer, device)
    acc, f1 = evaluate(model, train_loader, device)
    print(f"Epoch {epoch+1}: Loss={loss:.4f} | Acc={acc:.4f} | F1={f1:.4f}")

torch.save(model.state_dict(), "siamese_model_taskB.pth")


Epoch 1: Loss=0.1313 | Acc=0.9849 | F1=0.9311
Epoch 2: Loss=0.0539 | Acc=0.9896 | F1=0.9542
Epoch 3: Loss=0.0303 | Acc=0.9968 | F1=0.9860
Epoch 4: Loss=0.0279 | Acc=0.9959 | F1=0.9818
Epoch 5: Loss=0.0203 | Acc=0.9976 | F1=0.9896


In [ ]:

val_dataset = FacePairDataset(val_path, transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

acc, f1 = evaluate(model, val_loader, device)
print(f"Validation → Accuracy: {acc:.4f}, Macro-F1: {f1:.4f}")


Validation → Accuracy: 0.9504, Macro-F1: 0.8206


In [ ]:
!pip install torchviz

from torchviz import make_dot

model = SiameseNet().to('cpu')
x1 = torch.randn(1, 3, 224, 224)
x2 = torch.randn(1, 3, 224, 224)
out = model(x1, x2)

make_dot(out, params=dict(model.named_parameters())).render("siamese_model_diagram", format="png")

'siamese_model_diagram.png'